<a href="https://colab.research.google.com/github/miiikkee/n8n/blob/master/hazards_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#read the zipfile
import pandas as pd
df_zip = pd.read_excel('zipcode_county_data.xlsx')
print(df_zip.head())



   Zipcode      Type         City       County  County Code
0    90001  Standard  Los Angeles  Los Angeles         6037
1    90002  Standard  Los Angeles  Los Angeles         6037
2    90003  Standard  Los Angeles  Los Angeles         6037
3    90004  Standard  Los Angeles  Los Angeles         6037
4    90005  Standard  Los Angeles  Los Angeles         6037


In [6]:
#install the tools
!pip install selenium
!apt-get update
!apt-get install -y chromium-chromedriver
import sys
sys.path.append('/usr/lib/chromium-browser/chromedriver')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 133.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 42.7 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy/main amd64 Packages [47.4 kB]
Get:11 https://r2u.stat.illinois.e

In [5]:
#import function
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time


In [11]:
#Read excel
df_zip = pd.read_excel('zipcode_county_data.xlsx')
print(df_zip.head())
zip_codes = df_zip['Zipcode'].astype(str).tolist()

   Zipcode      Type         City       County  County Code
0    90001  Standard  Los Angeles  Los Angeles         6037
1    90002  Standard  Los Angeles  Los Angeles         6037
2    90003  Standard  Los Angeles  Los Angeles         6037
3    90004  Standard  Los Angeles  Los Angeles         6037
4    90005  Standard  Los Angeles  Los Angeles         6037


In [12]:
#Crawler
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=chrome_options)


results = []

#For every zipcode
for zip_code in zip_codes:
    print(f"🚀 正在处理 ZIP: {zip_code}")

    try:
        driver.get("https://myhazards.caloes.ca.gov/")
        time.sleep(5)

        # Close the window Modal
        try:
            close_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Close') or contains(text(), 'OK')]")
            close_button.click()
            print("✅ Window Closed")
            time.sleep(2)
        except Exception:
            print("⚠️ No popover. Go ahead")

        wait = WebDriverWait(driver, 20)
        search_box = wait.until(EC.element_to_be_clickable((By.ID, "address")))
        search_button = wait.until(EC.element_to_be_clickable((By.ID, "searchId")))

        search_box.clear()
        search_box.send_keys(zip_code)
        search_button.click()

        wait.until(EC.presence_of_element_located((By.ID, "searchResult")))
        time.sleep(5)

        # City Name
        try:
            map_text_element = wait.until(EC.presence_of_element_located((By.TAG_NAME, "text")))
            location_text = map_text_element.text.strip()
            city_name = location_text.split(",")[1].strip() if "," in location_text else "Unknown"
            print(f"Success: {city_name}")
        except Exception:
            city_name = "Unknown"

        # County name（from excel）
        try:
            county_name = df_zip[df_zip['zip_code'] == int(zip_code)]['county'].values[0]
        except Exception:
            county_name = "Unknown"

        # 提取灾害信息
        result_div = driver.find_element(By.ID, "searchResult")
        hazard_titles = result_div.find_elements(By.TAG_NAME, "h4")
        hazard_descriptions = result_div.find_elements(By.TAG_NAME, "h3")

        # 组装成一个字典
        entry = {
            "zip_code": zip_code,
            "county": county_name,
            "city": city_name
        }

        # 加灾害标题
        for i, title in enumerate(hazard_titles):
            entry[f"hazard_{i+1}_title"] = title.text.strip()

        # 加灾害描述
        for i, desc in enumerate(hazard_descriptions):
            entry[f"hazard_{i+1}_desc"] = desc.text.strip()

        results.append(entry)

    except Exception as e:
        print(f"❌ 抓取 {zip_code} 失败，错误: {e}")
        continue

# 抓完了，退出浏览器
driver.quit()


流式输出内容被截断，只能显示最后 5000 行内容。
🚀 正在处理 ZIP: 90047
⚠️ No popover. Go ahead
Success: Los Angeles
🚀 正在处理 ZIP: 90048
⚠️ No popover. Go ahead
Success: Los Angeles
🚀 正在处理 ZIP: 90049
⚠️ No popover. Go ahead
Success: Los Angeles
🚀 正在处理 ZIP: 90056
⚠️ No popover. Go ahead
Success: Los Angeles
🚀 正在处理 ZIP: 90057
⚠️ No popover. Go ahead
Success: Los Angeles
🚀 正在处理 ZIP: 90058
⚠️ No popover. Go ahead
Success: Los Angeles
🚀 正在处理 ZIP: 90059
⚠️ No popover. Go ahead
Success: Los Angeles
🚀 正在处理 ZIP: 90061
⚠️ No popover. Go ahead
Success: Los Angeles
🚀 正在处理 ZIP: 90062
⚠️ No popover. Go ahead
Success: Los Angeles
🚀 正在处理 ZIP: 90063
⚠️ No popover. Go ahead
Success: Los Angeles
🚀 正在处理 ZIP: 90064
⚠️ No popover. Go ahead
Success: Los Angeles
🚀 正在处理 ZIP: 90065
⚠️ No popover. Go ahead
Success: Los Angeles
🚀 正在处理 ZIP: 90066
⚠️ No popover. Go ahead
Success: Los Angeles
🚀 正在处理 ZIP: 90067
⚠️ No popover. Go ahead
Success: Los Angeles
🚀 正在处理 ZIP: 90068
⚠️ No popover. Go ahead
Success: Los Angeles
🚀 正在处理 ZIP: 90069
⚠️ No popo

In [2]:
import os
print(os.listdir('/content'))

['.config', 'sample_data']


In [7]:
# 导入模块
import pandas as pd
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 读取所有zipcode
df_zip = pd.read_excel('zipcode_county_data.xlsx')
zip_codes = df_zip['Zipcode'].astype(str).tolist()

# 读取中间文件，如果有的话
if os.path.exists('/content/intermediate_result.csv'):
    df_done = pd.read_csv('/content/intermediate_result.csv')
    finished_zip_codes = df_done['zip_code'].astype(str).tolist()
    print(f"🔄 检测到已有{len(finished_zip_codes)}条已完成数据，继续续跑")
else:
    df_done = pd.DataFrame()
    finished_zip_codes = []
    print(f"🆕 没有检测到中间文件，准备从头开始")

# 计算剩余要处理的
remaining_zip_codes = [z for z in zip_codes if z not in finished_zip_codes]
print(f"✅ 还有 {len(remaining_zip_codes)} 个ZIP待处理")


🆕 没有检测到中间文件，准备从头开始
✅ 还有 1711 个ZIP待处理


In [8]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 20)


In [9]:
results = []
failed_zips = []

save_every = 50  # 每50个保存一次

for idx, zip_code in enumerate(remaining_zip_codes, 1):
    print(f"🚀 正在处理 ZIP: {zip_code}")

    try:
        driver.get("https://myhazards.caloes.ca.gov/")
        time.sleep(5)

        # 关闭弹窗
        try:
            close_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Close') or contains(text(), 'OK')]")))
            close_button.click()
            print("✅ 关闭弹窗成功")
        except:
            print("⚠️ 没有弹窗，继续")

        # 输入地址
        search_box = wait.until(EC.element_to_be_clickable((By.ID, "address")))
        search_button = wait.until(EC.element_to_be_clickable((By.ID, "searchId")))

        search_box.clear()
        search_box.send_keys(zip_code)
        search_button.click()

        time.sleep(5)  # 等待页面加载

        # 获取City名
        try:
            map_text_element = wait.until(EC.presence_of_element_located((By.TAG_NAME, "text")))
            location_text = map_text_element.text.strip()
            city_name = location_text.split(",")[1].strip() if "," in location_text else "Unknown"
        except Exception as e:
            city_name = "Unknown"

        # 获取County名（从你的df_zip表格）
        try:
            county_name = df_zip[df_zip['Zipcode'] == int(zip_code)]['County'].values[0]
        except:
            county_name = "Unknown"

        # 获取风险信息
        result_div = driver.find_element(By.ID, "searchResult")
        hazard_titles = result_div.find_elements(By.TAG_NAME, "h4")
        hazard_descriptions = result_div.find_elements(By.TAG_NAME, "h3")

        # 解析每个Hazard
        entry = {
            "zip_code": zip_code,
            "city": city_name,
            "county": county_name
        }
        for i, desc in enumerate(hazard_descriptions):
            entry[f"hazard_{i+1}_desc"] = desc.text.strip()

        results.append(entry)
        print(f"✅ 完成 ZIP: {zip_code}")

    except Exception as e:
        print(f"❌ ZIP: {zip_code} 处理失败，错误：{e}")
        failed_zips.append(zip_code)
        continue

    # 每50个保存一次
    if idx % save_every == 0:
        pd.DataFrame(results).to_csv('/content/intermediate_result.csv', index=False)
        print(f"💾 已保存中间数据 {idx} 条")

# 全部完成后，保存完整数据
final_df = pd.concat([df_done, pd.DataFrame(results)], ignore_index=True)
final_df.to_csv('/content/final_hazard_result.csv', index=False)
print("🎉 全部数据已保存为 final_hazard_result.csv")

# 保存失败的ZIP
if failed_zips:
    pd.DataFrame({'failed_zip': failed_zips}).to_csv('/content/failed_zips.csv', index=False)
    print(f"⚠️ 有 {len(failed_zips)} 个ZIP失败，已保存到 failed_zips.csv")

driver.quit()


流式输出内容被截断，只能显示最后 5000 行内容。
🚀 正在处理 ZIP: 90066
⚠️ 没有弹窗，继续
✅ 完成 ZIP: 90066
🚀 正在处理 ZIP: 90067
⚠️ 没有弹窗，继续
✅ 完成 ZIP: 90067
🚀 正在处理 ZIP: 90068
⚠️ 没有弹窗，继续
✅ 完成 ZIP: 90068
🚀 正在处理 ZIP: 90069
⚠️ 没有弹窗，继续
✅ 完成 ZIP: 90069
🚀 正在处理 ZIP: 90071
⚠️ 没有弹窗，继续
✅ 完成 ZIP: 90071
🚀 正在处理 ZIP: 90073
⚠️ 没有弹窗，继续
✅ 完成 ZIP: 90073
🚀 正在处理 ZIP: 90077
⚠️ 没有弹窗，继续
✅ 完成 ZIP: 90077
🚀 正在处理 ZIP: 90079
⚠️ 没有弹窗，继续
✅ 完成 ZIP: 90079
🚀 正在处理 ZIP: 90089
⚠️ 没有弹窗，继续
✅ 完成 ZIP: 90089
🚀 正在处理 ZIP: 90090
⚠️ 没有弹窗，继续
✅ 完成 ZIP: 90090
🚀 正在处理 ZIP: 90094
⚠️ 没有弹窗，继续
✅ 完成 ZIP: 90094
🚀 正在处理 ZIP: 90095
⚠️ 没有弹窗，继续
✅ 完成 ZIP: 90095
🚀 正在处理 ZIP: 90201
⚠️ 没有弹窗，继续
✅ 完成 ZIP: 90201
🚀 正在处理 ZIP: 90210
⚠️ 没有弹窗，继续
✅ 完成 ZIP: 90210
🚀 正在处理 ZIP: 90211
⚠️ 没有弹窗，继续
✅ 完成 ZIP: 90211
🚀 正在处理 ZIP: 90212
⚠️ 没有弹窗，继续
✅ 完成 ZIP: 90212
🚀 正在处理 ZIP: 90220
⚠️ 没有弹窗，继续
✅ 完成 ZIP: 90220
🚀 正在处理 ZIP: 90221
⚠️ 没有弹窗，继续
✅ 完成 ZIP: 90221
🚀 正在处理 ZIP: 90222
⚠️ 没有弹窗，继续
✅ 完成 ZIP: 90222
🚀 正在处理 ZIP: 90230
⚠️ 没有弹窗，继续
✅ 完成 ZIP: 90230
🚀 正在处理 ZIP: 90232
⚠️ 没有弹窗，继续
✅ 完成 ZIP: 90232
🚀 正在处理 ZIP: 90240
⚠️ 没有弹窗，继续

# 新段落